In [26]:
import os
import pathlib
from glob import glob
import geopandas as gpd
import pandas as pd
import requests
import time

In [6]:
# Create data directory in the home folder
data_dir = os.path.join(
    # Home directory
    pathlib.Path.home(),
    # Earth analytics data directory
    'earth-analytics',
    'data',
    # Project directory
    'Pika_distribution',
)
#data_dir
os.makedirs(data_dir, exist_ok=True)

In [7]:
# Set up the ecoregion boundary URL
eco_url = "https://storage.googleapis.com/teow2016/Ecoregions2017.zip"

# Set up a path to save the data on your machine
re_dir = os.path.join(data_dir, 'resolve_ecoregions')
# Make the ecoregions directory

#Make the shape file directory. 
os.makedirs(re_dir, exist_ok=True)

# Join ecoregions shapefile path
eco_shp_path = os.path.join(re_dir, 'ecoregions.shp')

# Only download once
if not os.path.exists(eco_shp_path):
    eco_gdf = gpd.read_file(eco_url)
    eco_gdf.to_file(eco_shp_path)

In [8]:
%%bash
find ~/earth-analytics/data/Pika_distribution/resolve_ecoregions -name '*.shp' 

/home/jovyan/earth-analytics/data/Pika_distribution/resolve_ecoregions/ecoregions.shp


In [9]:
# Open up the ecoregions boundaries
eco_gdf = (
    gpd.read_file(eco_shp_path)
    [['OBJECTID', 'ECO_NAME', 'SHAPE_AREA', 'geometry']]
    .rename(columns={
        'OBJECTID': 'ecoregion_id',
        'ECO_NAME': 'name',
        'SHAPE_AREA': 'area'
    })
    .set_index('ecoregion_id')
)


In [10]:
eco_gdf

,name,area,geometry
ecoregion_id,,,
1.0,Adelie Land tundra,0.038948,"MULTIPOLYGON (((158.7141 -69.60657, 158.71264 ..."
2.0,Admiralty Islands lowland rain forests,0.170599,"MULTIPOLYGON (((147.28819 -2.57589, 147.2715 -..."
3.0,Aegean and Western Turkey sclerophyllous and m...,13.844952,"MULTIPOLYGON (((26.88659 35.32161, 26.88297 35..."
4.0,Afghan Mountains semi-desert,1.355536,"MULTIPOLYGON (((65.48655 34.71401, 65.52872 34..."
5.0,Ahklun and Kilbuck Upland Tundra,8.196573,"MULTIPOLYGON (((-160.26404 58.64097, -160.2673..."
...,...,...,...
848.0,Sulawesi lowland rain forests,9.422097,"MULTIPOLYGON (((117.33111 -7.53306, 117.30525 ..."
212.0,East African montane forests,5.010930,"MULTIPOLYGON (((36.7375 -3.13, 36.7375 -3.1316..."
224.0,Eastern Arc forests,0.890325,"MULTIPOLYGON (((36.38 -8.96583, 36.38 -8.96667..."


In [66]:
#Locate and define the gbif datafram
data_dir = os.path.join(
    # Home directory
    pathlib.Path.home(),
    # Earth analytics data directory
    'earth-analytics',
    'data',
    # Project directory
    'pika_distribution',
)
os.makedirs(data_dir, exist_ok=True)

# Define the directory name for GBIF data
gbif_dir = os.path.join(data_dir, 'gbif_pika_data')
gbif_pattern = os.path.join(gbif_dir, '*.csv')
gbif_path = glob(gbif_pattern)[0]

# Load the GBIF data
gbif_df = pd.read_csv(
    gbif_path, 
    delimiter='\t',
    index_col='gbifID',
    usecols=['gbifID', 'month', 'year', 'decimalLatitude', 'decimalLongitude', 'elevation']
)
gbif_df.head()

,decimalLatitude,decimalLongitude,elevation,month,year
gbifID,,,,,
923923961,40.587348,-111.617097,NaN,6.0,2014
923922049,40.543289,-111.692180,NaN,6.0,2014
923921599,37.912662,-119.265777,NaN,6.0,2014
911507490,48.000988,-121.424204,NaN,6.0,2014
911496161,47.751074,-120.740139,NaN,7.0,1992


In [12]:
#Convert gbif data to a geodataframe
gbif_gdf = (
    gpd.GeoDataFrame(
        gbif_df, 
        geometry=gpd.points_from_xy(
            gbif_df.decimalLongitude, 
            gbif_df.decimalLatitude), 
        crs="EPSG:4326")
    # Select the desired columns
    [['month', 'geometry']]
)
gbif_gdf

,month,geometry
gbifID,,
923923961,6.0,POINT (-111.6171 40.58735)
923922049,6.0,POINT (-111.69218 40.54329)
923921599,6.0,POINT (-119.26578 37.91266)
911507490,6.0,POINT (-121.4242 48.00099)
911496161,7.0,POINT (-120.74014 47.75107)
...,...,...
1024188695,7.0,POINT (-122.19333 41.36361)
1024184546,7.0,POINT (-111.67624 40.54453)
1024183578,6.0,POINT (-107.77146 37.85114)


In [13]:
#Combine the ecoregions and gbif data
gbif_ecoregions_gdf = (
    eco_gdf
    # Match the CRS of the GBIF data and the ecoregions
    .to_crs(gbif_gdf.crs)
    # Find ecoregion for each observation
    .sjoin(
        gbif_gdf,
        how='inner', 
        predicate='contains')
)


gbif_ecoregions_gdf

,name,area,geometry,gbifID,month
ecoregion_id,,,,,
74.0,Blue Mountains forests,8.066966,"POLYGON ((-116.73108 46.26653, -116.72977 46.2...",4947687918,8.0
74.0,Blue Mountains forests,8.066966,"POLYGON ((-116.73108 46.26653, -116.72977 46.2...",2862645406,8.0
74.0,Blue Mountains forests,8.066966,"POLYGON ((-116.73108 46.26653, -116.72977 46.2...",3455593572,8.0
74.0,Blue Mountains forests,8.066966,"POLYGON ((-116.73108 46.26653, -116.72977 46.2...",1852482961,8.0
74.0,Blue Mountains forests,8.066966,"POLYGON ((-116.73108 46.26653, -116.72977 46.2...",1145266945,8.0
...,...,...,...,...,...
839.0,Northern Rockies conifer forests,35.905513,"POLYGON ((-119.99977 54.53117, -119.8914 54.45...",3996587399,9.0
839.0,Northern Rockies conifer forests,35.905513,"POLYGON ((-119.99977 54.53117, -119.8914 54.45...",4600166120,5.0
839.0,Northern Rockies conifer forests,35.905513,"POLYGON ((-119.99977 54.53117, -119.8914 54.45...",3385101965,8.0


In [14]:
gbif_df.decimalLatitude

gbifID
923923961     40.587348
923922049     40.543289
923921599     37.912662
911507490     48.000988
911496161     47.751074
                ...    
1024188695    41.363610
1024184546    40.544526
1024183578    37.851140
1019052694    40.615000
1019052661    40.615000
Name: decimalLatitude, Length: 7508, dtype: float64

In [67]:
#Round the lat and lon data to be used for elevation requests.
gbif_df_rounded = gbif_df 
gbif_df_rounded['decimalLatitude'] = (
    gbif_df_rounded['decimalLatitude']
    .round(2)
)

gbif_df_rounded['decimalLongitude'] = (
    gbif_df_rounded['decimalLongitude']
    .round(2)
)

print(gbif_df_rounded)


            decimalLatitude  decimalLongitude  elevation  month  year
gbifID                                                               
923923961             40.59           -111.62        NaN    6.0  2014
923922049             40.54           -111.69        NaN    6.0  2014
923921599             37.91           -119.27        NaN    6.0  2014
911507490             48.00           -121.42        NaN    6.0  2014
911496161             47.75           -120.74        NaN    7.0  1992
...                     ...               ...        ...    ...   ...
1024188695            41.36           -122.19        NaN    7.0  2014
1024184546            40.54           -111.68        NaN    7.0  2014
1024183578            37.85           -107.77        NaN    6.0  2014
1019052694            40.62           -115.38        NaN    8.0  1998
1019052661            40.62           -115.38        NaN    8.0  1998

[7508 rows x 5 columns]


In [74]:
locations_str = "|".join([
    f"{row['decimalLatitude']},{row['decimalLongitude']}"
    for _, row in df_chunk.iterrows()
])

elev_dataset = "srtm30m"

    # Open Topo Data API endpoint
url = (
    f"https://api.opentopodata.org/v1/{elev_dataset}"
    f"?locations={locations_str}"
)


url

'https://api.opentopodata.org/v1/srtm30m?locations=40.59,-111.62|40.54,-111.69|37.91,-119.27|48.0,-121.42|47.75,-120.74|44.76,-121.79|44.94,-110.72|46.91,-121.64|37.85,-107.97|37.85,-107.97|37.85,-107.94|37.85,-107.97|37.85,-107.94|37.85,-107.97|37.85,-107.94|37.85,-107.97|37.85,-107.94|37.85,-107.97|37.85,-107.94|37.85,-107.97|37.85,-107.94|37.85,-107.94|37.84,-107.97|37.84,-107.94|37.84,-107.94|37.84,-107.97|37.86,-107.97|37.84,-107.97|37.83,-107.95|37.86,-107.97|37.83,-107.95|37.85,-107.99|37.86,-107.98|37.83,-107.95|37.86,-107.98|37.86,-107.98|37.83,-107.95|39.23,-106.17|37.83,-107.9|37.83,-107.91|37.83,-107.9|37.77,-107.58|37.76,-107.58|37.57,-107.23|37.59,-107.23|37.57,-107.24|37.63,-107.27|37.63,-107.27|37.6,-107.34|37.61,-107.24|37.63,-107.44|37.63,-107.2|37.61,-107.23|37.67,-107.37|37.63,-107.19|37.64,-107.28|37.61,-107.21|37.63,-107.18|37.74,-107.85|37.69,-107.52|37.74,-107.84|37.76,-107.8|37.74,-107.83|37.76,-107.79|37.68,-107.98|37.69,-107.98|37.75,-107.82|37.77,-107.97|37.

In [78]:
# Define a function to pull elevation data via Open-Elevation API
def get_altitudes(df):
    # Prepare the locations as a string
    locations_str = (
        "|".join([
            f"{row['decimalLatitude']},{row['decimalLongitude']}" 
            for _, row in df.iterrows()
        ])
    )

    # Dataset resolution and location
    elev_dataset = "srtm30m"

    # Open Topo Data API endpoint
    url = (
        f"https://api.opentopodata.org/v1/{elev_dataset}"
        f"?locations={locations_str}"
    )

    # Make a request to the Open Topo Data API
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the response JSON data
        data = response.json()
        
        # Add the elevation data to the DataFrame
        df['elevation'] = [result['elevation'] for result in data['results']]
        return df
    else:
        print(f"Failed to connect, status code: {response.status_code}")
        return None

In [81]:
#Define a function to split the dataset into chuncks of 100 per request
def split_dataframe(gbif_df, chunk_size=100):
    for i in range(0, gbif_df.shape[0], chunk_size):
    #for i in range(0, 99, chunk_size):
        yield gbif_df.iloc[i:i + chunk_size]

chunk_list = []

# Example usage to process data in chunks
for df_chunk in split_dataframe(gbif_df, chunk_size=100):
    df_chunk_with_altitudes = get_altitudes(df_chunk)



#Append the processed chunk to the list
    chunk_list.append(df_chunk_with_altitudes)
    
    # Optional: add a delay between API requests to avoid rate limiting
    time.sleep(2)

gbif_df_wAltitude = pd.concat(chunk_list, ignore_index=True)


/tmp/ipykernel_1205/1013217777.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['elevation'] = [result['elevation'] for result in data['results']]


In [82]:
gbif_df_wAltitude


,decimalLatitude,decimalLongitude,elevation,month,year
0,40.59,-111.62,2781.0,6.0,2014
1,40.54,-111.69,3049.0,6.0,2014
2,37.91,-119.27,3152.0,6.0,2014
3,48.00,-121.42,816.0,6.0,2014
4,47.75,-120.74,620.0,7.0,1992
...,...,...,...,...,...
7503,41.36,-122.19,2427.0,7.0,2014
7504,40.54,-111.68,3041.0,7.0,2014
7505,37.85,-107.77,3516.0,6.0,2014
7506,40.62,-115.38,3033.0,8.0,1998
